# <span style="color: SlateBlue;">Contexto</span> 

Contiene información sobre diferentes atributos relacionados con viviendas en varios barrios de Boston, Massachusetts, Estados Unidos. El objetivo principal de este conjunto de datos es predecir el valor medio de las viviendas en esos barrios en función de las características proporcionadas.

Atributos:
   1. CRIM: Tasa de criminalidad per cápita por barrio.
   2. ZN: Proporción de tierra residencial dividida en zonas para lotes grandes.
   3. INDUS: Proporción de acres de negocios no minoristas por barrio.
   4. CHAS: Variable ficticia de Charles River (1 si limita con el río, 0 en caso contrario).
   5. NOX: Concentración de óxidos de nitrógeno (partes por 10 millones).
   6. RM: Número promedio de habitaciones por vivienda.
   7. AGE: Proporción de viviendas ocupadas por sus propietarios construidas antes de 1940.
   8. DIS: Distancia ponderada a los centros de empleo en Boston.
   9. RAD: Índice de accesibilidad a carreteras radiales.
   10. TAX: Tasa de impuesto a la propiedad.
   11. PTRATIO: Relación alumno-profesor en las escuelas locales.
   12. B: Proporción de residentes afroamericanos por barrio.
   13. LSTAT: Porcentaje de población de bajos ingresos.
   14. MEDV: Valor medio de las viviendas ocupadas por sus propietarios en miles de dólares.

# <span style="color: SlateBlue;">Objetivo</span> 

Desarrollar y evaluar modelos de redes neuronales para un problema de regresión. 

Partiremos de una red neuronal de referencia y buscaremos preparar los datos y optimizar la topología para mejorar los resultados obtenidos.

# <span style="color: SlateBlue;">Importar clases y funciones</span> 

In [1]:
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from scikeras.wrappers import KerasRegressor
from sklearn.model_selection import cross_val_score, KFold

## Obtención y división del dataset

In [2]:
df = pd.read_csv('../Datasets/housing.csv', delim_whitespace=True, header=None)

In [3]:
dataset = df.values

X = dataset[:, 0:13]
y = dataset[:, 13]

# <span style="color: SlateBlue;">Modelo de referencia</span> 

Nuestro modelo de referencia estará conformado por:

   - Una única capa oculta completamente conectada con el mismo número de neuronas que los atributos de entrada (13)
   - Función de activacion ReLu en la capa oculta
   - ADAM como algoritmo de optimización y el MSE como función de pérdida
   - El objeto wrapper para el modelo será un `KerasRegressor`
   - Evaluamos el modelo con 10-fold

In [4]:
def baseline_model():
    
    model = Sequential()
    
    model.add(Dense(13, 
                    input_dim=13,
                    activation='relu'))
    
    model.add(Dense(1))
    
    model.compile(loss='mean_squared_error',
                 optimizer='adam')
    
    return model

In [5]:
estimator = KerasRegressor(model=baseline_model,
                          epochs=100,
                          batch_size=5,
                          verbose=0)

kfold = KFold(n_splits=10,
             shuffle=True)

results = cross_val_score(estimator,
                         X,
                         y,
                         cv=kfold)

In [6]:
print('Modelo base: %.2f (%.2f) MSE' % (results.mean(), results.std()))

Modelo base: 0.58 (0.11) MSE


# <span style="color: SlateBlue;">Optimización por procesamiento de datos</span> 

In [7]:
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

In [8]:
estimators = []
estimators.append(('standarize', StandardScaler()))
estimators.append(('NN', KerasRegressor(model=baseline_model,
                                       epochs=100,
                                       batch_size=5,
                                       verbose=0)))
pipeline = Pipeline(estimators)

kfold = KFold(n_splits=10,
             shuffle=True)

results = cross_val_score(pipeline,
                         X,
                         y,
                         cv=kfold)

In [9]:
print('Modelo estandarizado: %.2f (%.2f) MSE' % (results.mean(), results.std()))

Modelo estandarizado: 0.83 (0.10) MSE


# <span style="color: SlateBlue;">Ajuste de la topología</span> 

## Topología más profunda

Una posible forma de mejorar la red neuronal es hacerla más profunda, es decir, añadiendo más capas. En este caso vamos a utilizar una segunda capa oculta con aproximadamente la mitad de neuronas. Es decir:
```
            13 inputs -> [13 neuronas -> 6 neuronas] -> 1 output
```

Entonces, vamos a evaluar este modelo aplicando la estandarización anterior

In [11]:
def deeper_model():
    
    model = Sequential()
    
    model.add(Dense(13, 
                    input_dim=13,
                    activation='relu'))
    model.add(Dense(6,
                   activation='relu'))
    model.add(Dense(1))
    
    model.compile(loss='mean_squared_error',
                 optimizer='adam')
    
    return model

In [19]:
estimators = []
estimators.append(('standarize', StandardScaler()))
estimators.append(('Deep-NN', KerasRegressor(model=deeper_model,
                                       epochs=100,
                                       batch_size=5,
                                       verbose=0)))
pipeline = Pipeline(estimators)

kfold = KFold(n_splits=10,
             shuffle=True)

results = cross_val_score(pipeline,
                         X,
                         y,
                         cv=kfold)

In [20]:
print('Deep-NN+Estandarizado: %.2f (%.2f) MSE' % (results.mean(), results.std()))

Deep-NN+Estandarizado: 0.80 (0.17) MSE


## Topología más ancha

Otra posible opción de mejorar la red neuronal es hacerla más amplia. En este caso vamos a utilizar 20 neuronas en la capa oculta. Es decir:
```
            13 inputs -> 20 neuronas -> 1 output
```


In [23]:
def wider_model():
    
    model = Sequential()
    
    model.add(Dense(20, 
                    input_dim=13,
                    activation='relu'))
    model.add(Dense(1))
    
    model.compile(loss='mean_squared_error',
                 optimizer='adam')
    
    return model

In [24]:
estimators = []
estimators.append(('standarize', StandardScaler()))
estimators.append(('Wider-NN', KerasRegressor(model=wider_model,
                                       epochs=100,
                                       batch_size=5,
                                       verbose=0)))
pipeline = Pipeline(estimators)

kfold = KFold(n_splits=10,
             shuffle=True)

results = cross_val_score(pipeline,
                         X,
                         y,
                         cv=kfold)

In [25]:
print('Wider-NN+Estandarizado: %.2f (%.2f) MSE' % (results.mean(), results.std()))

Wider-NN+Estandarizado: 0.84 (0.06) MSE
